# Two Newsvendor Problem

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/stochastic-optimization/blob/master/notebooks/two_newsvendor.ipynb)

In [ ]:
!pip install -q jax jaxlib jaxtyping chex matplotlib
import os
if 'COLAB_GPU' in os.environ or not os.path.exists('problems'):
    !git clone https://github.com/pedronahum/stochastic-optimization.git
    os.chdir('stochastic-optimization')

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from problems.two_newsvendor import TwoNewsvendorConfig, TwoNewsvendorModel, NewsvendorFieldPolicy

In [ ]:
config = TwoNewsvendorConfig(n_newsvendors=2, horizon=30)
model = TwoNewsvendorModel(config)
policy = NewsvendorFieldPolicy()
key = jax.random.PRNGKey(42)
state = model.init_state(key)
print('✓ Two newsvendor model ready')

In [ ]:
# Simulate coordination
orders, allocations, rewards = [], [], []
for t in range(30):
    key, k1, k2 = jax.random.split(key, 3)
    decision = policy(None, state, k1, model)
    exog = model.sample_exogenous(k2, state, t)
    reward = model.reward(state, decision, exog)
    orders.append(decision[:2].tolist())
    allocations.append(decision[2] if len(decision) > 2 else 0)
    rewards.append(float(reward))
    state = model.transition(state, decision, exog)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot([o[0] for o in orders], label='Newsvendor 1')
plt.plot([o[1] for o in orders], label='Newsvendor 2')
plt.title('Order Quantities')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(np.cumsum(rewards))
plt.title('Cumulative Profit')
plt.tight_layout()
plt.show()